In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gpplot as gpp
import csv, requests, warnings, os, urllib3, sys, hashlib, subprocess
sys.path.append('../scripts/')
import download_demult_files as down
warnings.filterwarnings("ignore")

In [40]:
def get_inputs():
    username = input("Enter the username provided by WalkUp:")
    password = input("Enter the password provided by WalkUp:")
    url = input("Enter the URL provided by WalkUp:")
    outputname = input("Enter the name of the output folder to save demultiplexed files in")
    outputpath = input("Path to output folder to save demultiplexed files in (Do not include the name of the folder itself)')")
    return username, password, url, outputname, outputpath

def download_locally():
    username, password, url, outputname, outputpath = get_inputs()
    dir_cmd = 'mkdir '+outputpath+'/'+outputname
    print('Creating directory...')
    os.system(dir_cmd)
    cmd = 'wget --tries=10 --continue --mirror --user '+ username +' --password '+ password +\
    ' --no-check-certificate '+ url +' -P '+outputpath+'/'+outputname
    try:
        sys.path.append('/usr/local/bin/')
        print('Downloading files...')
        os.system(cmd)
        print('Files downloaded...')
    except:
        sys.exit()
    return outputname, outputpath, username

# def upload_Gdrive(in_path, outputname, username):
#     out_path = "/Volumes/GoogleDrive/Shared drives/GPP Cloud /R&D/Sequencing_Backup/"
#     exist_folders = os.listdir(out_path)
#     if outputname not in exist_folders:
#         cmd = 'mkdir \"'+out_path+outputname+'\"'
#         try:
#             print('Creating directory...')
#             os.system(cmd)
#         except:
#             sys.exit('Error creating output folder')
#     else:
#         print('Folder exists; adding files to it')
#     cmd = 'cp -r \"'+in_path+outputname+'\" \"'+out_path+outputname+'\"'
#     try:
#         print('Backing up files...')
#         os.system(cmd)
#         val = check_md5(in_path,outputname,out_path,username)
#         print('Files moved and MD5 checksums verified')
#     except:
#         sys.exit('Error transferring files')
#     return

def upload_Gdrive(in_path, outputname, username):
    out_path = "R&D/Sequencing_Backup/2020/"
    cmd = 'rclone copy \"'+in_path+outputname+'\" remote:\"'+out_path+outputname+'\"'
    try:
        print('Backing up files...')
        os.system(cmd)
        val = check_md5(in_path,outputname,'/Volumes/GoogleDrive/Shared drives/GPP Cloud /'+out_path,username)
        print('Files moved and MD5 checksums verified')
    except:
        sys.exit('Error transferring files')
    return

def check_md5(in_path, foldername, out_path, username):
    in_folder = in_path+'/'+foldername+'/get.broadinstitute.org/pkgs/'+username
    out_folder = out_path+'/'+foldername+'/get.broadinstitute.org/pkgs/'+username
    in_files = os.listdir(in_folder)
    in_files = [x for x in in_files if 'fastq' in x]
    print(in_files)
    #out_files = os.listdir(out_folder)
    for f in in_files:
        print(f)
        #out_file = [x for x in out_files if x==f][0]
        try:
            assert(hashlib.md5(open(in_folder+'/'+f,'rb').read()).hexdigest() == hashlib.md5(open(out_folder+'/'+f,'rb').read()).hexdigest())
        except AssertionError:
            print('MD5 checksums do not match;'+f)
    return 

def run_fastqc():
    fastqc_in_folder = input('Please enter the path to the folder with files for FASTQC:')
    fastqc_out_folder = input('Please enter the name of the output folder:')
    fastqc_in_files = os.listdir(fastqc_in_folder)
    fastqc_in_files = [x for x in fastqc_in_files if ('fastq' in x)&('barcode' not in x)]
    for f in fastqc_in_files:
        cmd = 'mkdir "/Volumes/GoogleDrive/Shared drives/GPP Cloud /R&D/FASTQC_outputs/2020/'+fastqc_out_folder+'"'
        value = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
        cmd = 'fastqc "'+fastqc_in_folder+'/'+f+'" -o "/Volumes/GoogleDrive/Shared drives/GPP Cloud /R&D/FASTQC_outputs/2020/'+fastqc_out_folder+'"'
        value = subprocess.check_output(cmd, stderr=subprocess.STDOUT)
    return

This notebook will help set up input files for running CRISPResso. If you are running this notebook for the first time, before running this notebook, please make sure you:
<ol>
    <li>Install Homebrew on your system by pasting the following command on your terminal:
    <b>/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"</b></li>
    <li>Once Homebrew is installed, install wget by typing <b>brew install wget</b> on the terminal. wget helps download files from the WalkUp sequencing server.</li>
</ol>

## Downloading sequencing files

We will first download files on to our machine. 

In [6]:
outputname, outputpath, username = download_locally()

Enter the username provided by WalkUp:SN0214446
Enter the password provided by WalkUp:p5aermB7kAgg
Enter the URL provided by WalkUp:https://get.broadinstitute.org/pkgs/SN0214446/
Enter the name of the output folder to save demultiplexed files indownload_test_0202
Path to output folder to save demultiplexed files in (Do not include the name of the folder itself)')/Users/mhegde/Desktop/file_download_test/
Creating directory...
Files downloaded...


## Backing up on GDrive

In [23]:
upload_Gdrive(outputpath, outputname, username)

Backing up files...
['1_JCV86.1.barcode_1.fastq.gz', '1_JCV86.1.1.fastq.gz']
1_JCV86.1.barcode_1.fastq.gz
1_JCV86.1.1.fastq.gz
Files moved and MD5 checksums verified


In [35]:
# gauth = GoogleAuth()
# gauth.LocalWebserverAuth()
# drive = GoogleDrive(gauth)

In [64]:
# fileList = drive.ListFile({'q':"'1uMXOLjvfY9TNlhwj0fVcTp6k2-heQe0c' in parents and trashed=false", 
#                            'corpora': 'teamDrive', 
#                            'teamDriveId': '0ABXsHzY7MltDUk9PVA', 
#                            'includeTeamDriveItems': True, 
#                            'supportsTeamDrives': True}).GetList()
# for file in fileList:
#     print('Title: %s, ID: %s' % (file['title'], file['id']))
#       # Get the folder ID that you want
#     if(file['title'] == "Sequencing_Backup"):
#         fileID = file['id']
#         print(fileID)

## Running FASTQC

In [41]:
run_fastqc()

Please enter the path to the folder with files for FASTQC:/Volumes/GoogleDrive/Shared drives/GPP Cloud /R&D/Sequencing_Backup/2020/NGBEV_MCL1/AKS_MCL1_NGBEV_D9:16/demultiplexed/
Please enter the name of the output folder:AKS_MCL1_NGBEV_D9_16


FileNotFoundError: [Errno 2] No such file or directory: 'mkdir "/Volumes/GoogleDrive/Shared drives/GPP Cloud /R&D/FASTQC_outputs/2020/AKS_MCL1_NGBEV_D9_16"'